In [92]:
import sys
sys.path.append('/Users/mat/Projects/PySyft')

In [7]:
%load_ext autoreload
%autoreload 2

# AutogradTensor Development

We have an issue with backprop in PyTorch. When you call `.backward()` on a torch.Tensor it immediately drops down into C++ code. However, we need to keep track of all operations in Python for use in calculating sensitivity as well as other things. We need to override `.backward()` such that we go through the backpropagation chain in Python instead of the C++ layer.

The idea here is we create a `BackpropTensor` that overrides `.backward()`. Maybe a different name, like `GradientTensor` since it should only be used on tensors where we need gradients. We can discuss naming later I suppose. Anyway, this new tensor should override `.backward()`...

In [2]:
import torch

In [ ]:
for grad, grad_fn in zip(grads, z.grad_fn.next_functions):
    grad_fn[0](grad)

In [11]:
grad_fn = GradAdd(1, 2)
grad_fn(torch.ones(1,2))

tensor([[1., 1.]])

In [70]:
class GradAdd:
    def __init__(self, *args):
        self.next_functions = []
        
    def __call__(self, grad):
        return grad # * 1

In [84]:
class Accumulate:
    def __init__(self, tensor):
        self.tensor = tensor
        self.next_functions = []
        
    def __call__(self, grad):
        if self.tensor.grad:
            self.tensor.grad += grad
        else:
            self.tensor.grad = grad + 0

In [85]:
def backwards_grad(grad_fn, in_grad=None):
    back_grad = grad_fn(in_grad)
    for next_grad_fn in grad_fn.next_functions:
        print(next_grad_fn)
        backwards_grad(next_grad_fn, back_grad)
    
class Tensor:
    def __init__(self, data, requires_grad=True):
        self.grad = None
        self.grad_fn = None
        self.child = data
        self.requires_grad = requires_grad
    
    def backward(self, grad=None):
        if grad is None:
            grad = torch.ones_like(self.child)
        backwards_grad(self.grad_fn, grad)
        
    def add(self, other):
        result = Tensor(self.child + other.child)
        
        grad_fn = GradAdd()
        grad_fn.next_functions = (Accumulate(self) if self.grad_fn is None else self.grad_fn, 
                                  Accumulate(other) if self.grad_fn is None else self.grad_fn)
        result.grad_fn = grad_fn
        
        return result

In [86]:
x = Tensor(torch.randn(3), requires_grad=True)
y = Tensor(torch.randn(3))

In [87]:
z = x.add(y)

In [88]:
x.grad

In [89]:
z.backward()

In [90]:
x.grad

tensor([1., 1., 1.])

In [34]:
w.grad_fn.next_functions

In [35]:
x = torch.rand((1, 3), requires_grad=True)
y = torch.rand((3, 1), requires_grad=True)

In [38]:
z.grad_fn.next_functions[0][0]

((<MmBackward at 0x122deaf60>, 0),)

In [36]:
z = torch.mm(x, y).mean()

In [6]:
z.grad_fn.next_functions

((<AccumulateGrad at 0x122da5fd0>, 0), (<AccumulateGrad at 0x122d2e048>, 0))

In [6]:
for grad, grad_fn in zip(grads, z.grad_fn.next_functions):
    grad_fn[0](grad)

In [13]:
torch.autograd.

<function torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False, only_inputs=True, allow_unused=False)>

In [52]:
# gradient of z w.r.t. y
grad_z_y = z.grad_fn(torch.ones(1))

In [56]:
# gradient of y w.r.t. x
grad_y_x = y.grad_fn(grad_z_y)

In [76]:
x.grad

In [77]:
def backwards_grad(grad_fn, in_grad=None):
    back_grad = grad_fn(in_grad)
    for next_grad_fn, _ in grad_fn.next_functions:
        print(next_grad_fn)
        backwards_grad(next_grad_fn, back_grad)

def backward(tensor):
    backwards_grad(tensor.grad_fn, torch.ones_like(tensor))

In [80]:
backward(z)

In [81]:
x.grad

tensor([1.3333, 2.6667, 4.0000], grad_fn=<AddBackward0>)

In [82]:
x

tensor([1., 2., 3.], requires_grad=True)

In [83]:
y

tensor([1., 4., 9.], grad_fn=<PowBackward0>)

In [88]:
w = torch.cat((x, y)).mean()

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('/Users/mat/Projects/PySyft')

# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import PointerTensor
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
from syft.frameworks.torch.tensors.decorators import LoggingTensor
import sys
import torch
hook = sy.TorchHook(torch)
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
a = torch.tensor([1,2.,3], requires_grad=False)

In [3]:
b = AutogradTensor().on(a)

Setting gradient


In [4]:
c = (b + b)

Setting gradient


In [5]:
c.backward()

My backward
AutogradTensor>tensor([1., 2., 3.])
Getting grad
Setting gradient
AutogradTensor>tensor([1., 2., 3.])
Getting grad
Getting grad
Setting gradient


In [7]:
b.child.grad

Getting grad


tensor([2., 2., 2.])

In [9]:
c.grad_fn.next_functions

(<syft.frameworks.torch.tensors.interpreters.gradients.Accumulate at 0x10a735828>,
 <syft.frameworks.torch.tensors.interpreters.gradients.Accumulate at 0x10a7357f0>)

In [12]:
b

(Wrapper)>AutogradTensor>tensor([1., 2., 3.])

In [4]:
bob = sy.VirtualWorker(hook, id="bob", verbose=True)

In [5]:
x = torch.tensor([1., 2., 3.], requires_grad=True).send(bob)
y = x**2
z = y.mean()

worker <VirtualWorker id:bob #tensors:0> received 2 tensor([1., 2., 3.], requires_grad=True)
worker <VirtualWorker id:bob #tensors:1> received 1 (b'__pow__', tensor([1., 2., 3.], requires_grad=True), (2,))
worker <VirtualWorker id:bob #tensors:2> received 1 (b'mean', tensor([1., 4., 9.], grad_fn=<PowBackward0>), ())


In [6]:
w = (x + y).mean()

worker <VirtualWorker id:bob #tensors:3> received 1 (b'__add__', tensor([1., 2., 3.], requires_grad=True), (tensor([1., 4., 9.], grad_fn=<PowBackward0>),))
worker <VirtualWorker id:bob #tensors:4> received 1 (b'mean', tensor([ 2.,  6., 12.], grad_fn=<AddBackward0>), ())
worker <VirtualWorker id:bob #tensors:5> received 4 72286273856


In [11]:
bob._objects

{53800654044: tensor([1., 2., 3.], requires_grad=True),
 28592088591: tensor([1., 4., 9.], grad_fn=<PowBackward0>),
 25244642045: tensor(4.6667, grad_fn=<MeanBackward1>),
 78768590182: tensor(6.6667, grad_fn=<MeanBackward1>),
 39043869483: tensor(1455.7036, grad_fn=<PowBackward0>)}

In [10]:
c = c**3

worker <VirtualWorker id:bob #tensors:5> received 1 (b'__pow__', tensor(11.3333, grad_fn=<AddBackward0>), (3,))
worker <VirtualWorker id:bob #tensors:6> received 4 77754589232


In [7]:
bob[z.id_at_location].grad_fn(torch.tensor(1.))

tensor([0.3333, 0.3333, 0.3333])

In [9]:
z.data

(Wrapper)>[PointerTensor | me:4881151385 -> bob:4881151385]::data

In [5]:
z.grad_fn(torch.ones_like(z))

worker <VirtualWorker id:bob #tensors:3> received 1 (b'torch.ones_like', None, (tensor(4.6667, grad_fn=<method>),))


AttributeError: 'PointerTensor' object has no attribute 'child'

In [4]:
print(x.requires_grad)
print(x.data.requires_grad)

False
False


In [15]:
bob._objects[z.child.id].grad_fn

In [20]:
z.child

[PointerTensor | me:18653888307 -> bob:18653888307]

In [10]:
z.backward()

Running backward_grad


TypeError: 'NoneType' object is not callable

In [12]:
z_local = z.get()